In [2]:
# The code was removed by DSX for sharing.

0     After waiting what seemed like  G52   forever...
1     Yeah   G15   it was such a change I  G40   sw...
2       G13  And I was so glad that all of the meas...
3     Ben and I had been saying for the past two we...
4     Of course   G20  that is what happened for th...
Name: sentence, dtype: object

In [3]:
class data():
    def __init__(self):
        self.data = []
        
class mini_data():
    def __init__(self):
        self.mini_data = []
        
d = data()

def read_data(sentence):
    md = mini_data()
    sentence = sentence.lower()
    content = sentence.split()
    new = md.mini_data + content
    d.data.append(new)

df_data['sentence'].apply(read_data)
training_data = np.array(d.data)

print('training_data length: ' + str(len(training_data)))


avrgs = [len(x) for x in training_data]
print('longest sentence: ' + str(np.max(avrgs)))
print('shortest sentence: ' + str(np.min(avrgs)))
print('avrg sentence length: ' + str(np.mean(avrgs)))
print('standard deviation of sentence length: ' + str(np.std(avrgs)))

'''sentence length will be the average length plus the standard deviation'''
sentence_length = round(np.mean(avrgs) + np.std(avrgs))
print('sentence length: ' + str(sentence_length))


training_data length: 874
longest sentence: 150
shortest sentence: 1
avrg sentence length: 27.0778032037
standard deviation of sentence length: 16.6001126717
sentence length: 44.0


In [4]:
import collections

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

words = [item for sublist in training_data for item in sublist]
dictionary, reverse_dictionary = build_dataset(words)
vocab_size = len(dictionary)

#print(dictionary)
print(vocab_size)
dictionary['the']

3277


0

In [5]:
train_input = []
train_output = []

def create_input_output(sentence):
    
    prev_gesture = False
    temp_out = [-1]*57
    
    '''create a 44-length array'''
    single_input = [[-1]]*int(sentence_length)
    single_output = [temp_out]*(int(sentence_length)+1)
    loc = 0
    
    for j in range(0,len(sentence)):
        
        '''if the sentence does still have words'''
        if j < int(sentence_length):
            
            word = sentence[j]
            #print(word)
            gesture = re.search(r'g\d{1,2}',word)
            
            if gesture is None:
                
                if prev_gesture:
                    
                    prev_gesture = False
                
                single_output[loc] = temp_out
                temp_out = [0]*57
                num_word = dictionary[word]

                single_input[loc] = [num_word]
                loc = loc + 1
                
            else:                    
                
                num_gesture = int(word[1:])
                del single_input[j]
                single_input.append([-1])
                temp_out[num_gesture] = 1
                prev_gesture = True
                
                
    train_input.append(single_input)
    train_output.append(single_output)
    return 'Done!'
        
#create_input_output(training_data[0])
ret_value = [create_input_output(sentence) for sentence in training_data]
print(len(train_input))
print(len(train_input[356]))
print(len(train_output))
print(len(train_output[48][34]))
#print(train_input[0])
#print(train_output[0][26])

874
44
874
57


In [6]:
print(train_input[0])
print(sentence_length)

[[69], [969], [112], [325], [68], [806], [2], [66], [155], [0], [278], [5], [487], [45], [2], [71], [31], [136], [47], [12], [0], [76], [61], [8], [572], [357], [1], [354], [112], [4], [499], [9], [7], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
44.0


In [225]:
NUM_EXAMPLES = 800
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:] #everything beyond 800
 
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 800

In [9]:
from tensorflow.contrib import rnn

num_units = 25
n_input = 4
time_steps = sentence_length / n_input
learning_rate = 0.001
n_classes = 57
batch_size = 50


In [10]:
out_weights = tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias = tf.Variable(tf.random_normal([n_classes]))

x = tf.placeholder("float", [None, time_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

In [11]:
input = tf.unstack(x, time_steps, 1)

In [12]:
lstm_layer = rnn.BasicLSTMCell(num_units, forget_bias=1)
outputs,_ = rnn.static_rnn(lstm_layer, input, dtype="float32")

In [13]:
prediction = tf.matmul(outputs[-1], out_weights) + out_bias